# 08.03 - Mapas Interactivos con Folium

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Intermedio  
**Tiempo estimado:** 25 min

---

## Que aprenderemos?

- Crear mapas interactivos con Folium
- Anadir marcadores y popups
- Capas y tiles
- Mapas de calor
- Exportar mapas HTML

In [1]:
import pandas as pd
import numpy as np

try:
    import folium
    from folium.plugins import HeatMap, MarkerCluster
    FOLIUM_AVAILABLE = True
    print(f"folium version: {folium.__version__}")
except ImportError:
    FOLIUM_AVAILABLE = False
    print("folium no instalado. pip install folium")

folium version: 0.20.0


---

## Datos de ejemplo

In [2]:
# Estaciones de bici en Madrid
stations = pd.DataFrame({
    'name': ['Sol', 'Atocha', 'Cibeles', 'Retiro', 'Gran Via', 
             'Lavapies', 'Opera', 'Tribunal', 'Alonso Martinez', 'Colon'],
    'lat': [40.4168, 40.4065, 40.4197, 40.4153, 40.4203,
            40.4095, 40.4180, 40.4260, 40.4280, 40.4250],
    'lon': [-3.7038, -3.6893, -3.6921, -3.6844, -3.7065,
            -3.7012, -3.7098, -3.7005, -3.6965, -3.6890],
    'bikes': [15, 22, 8, 12, 18, 10, 14, 9, 16, 20],
    'docks': [30, 35, 25, 28, 32, 24, 28, 22, 30, 35],
    'status': ['active', 'active', 'low', 'active', 'active',
               'low', 'active', 'low', 'active', 'active']
})

stations['available_pct'] = (stations['bikes'] / stations['docks'] * 100).round(0)
stations

,name,lat,lon,bikes,docks,status,available_pct
0,Sol,40.4168,-3.7038,15,30,active,50.0
1,Atocha,40.4065,-3.6893,22,35,active,63.0
2,Cibeles,40.4197,-3.6921,8,25,low,32.0
3,Retiro,40.4153,-3.6844,12,28,active,43.0
4,Gran Via,40.4203,-3.7065,18,32,active,56.0
5,Lavapies,40.4095,-3.7012,10,24,low,42.0
6,Opera,40.4180,-3.7098,14,28,active,50.0
7,Tribunal,40.4260,-3.7005,9,22,low,41.0
8,Alonso Martinez,40.4280,-3.6965,16,30,active,53.0
9,Colon,40.4250,-3.6890,20,35,active,57.0


---

## 1. Mapa basico

In [3]:
if FOLIUM_AVAILABLE:
    # Centro del mapa: Madrid
    madrid_coords = [40.4168, -3.7038]
    
    # Crear mapa
    m = folium.Map(
        location=madrid_coords,
        zoom_start=14
    )
    
    m

### Diferentes estilos de mapa (tiles)

In [4]:
if FOLIUM_AVAILABLE:
    # CartoDB Positron (limpio, minimalista)
    m = folium.Map(
        location=madrid_coords,
        zoom_start=14,
        tiles='CartoDB positron'
    )
    m

---

## 2. Anadir marcadores

In [5]:
if FOLIUM_AVAILABLE:
    m = folium.Map(location=madrid_coords, zoom_start=14, tiles='CartoDB positron')
    
    # Anadir marcador simple
    folium.Marker(
        location=[40.4168, -3.7038],
        popup='Puerta del Sol',
        tooltip='Click para info'
    ).add_to(m)
    
    m

In [6]:
if FOLIUM_AVAILABLE:
    m = folium.Map(location=madrid_coords, zoom_start=14, tiles='CartoDB positron')
    
    # Marcadores para todas las estaciones
    for idx, row in stations.iterrows():
        # Color segun disponibilidad
        if row['status'] == 'low':
            color = 'red'
            icon = 'exclamation-sign'
        else:
            color = 'green'
            icon = 'ok-sign'
        
        # Popup con HTML
        popup_html = f"""
        <div style="width: 150px">
            <h4>{row['name']}</h4>
            <b>Bicis:</b> {row['bikes']}/{row['docks']}<br>
            <b>Disponible:</b> {row['available_pct']:.0f}%
        </div>
        """
        
        folium.Marker(
            location=[row['lat'], row['lon']],
            popup=folium.Popup(popup_html, max_width=200),
            tooltip=row['name'],
            icon=folium.Icon(color=color, icon=icon)
        ).add_to(m)
    
    m

---

## 3. CircleMarker (tamano variable)

In [7]:
if FOLIUM_AVAILABLE:
    m = folium.Map(location=madrid_coords, zoom_start=14, tiles='CartoDB positron')
    
    for idx, row in stations.iterrows():
        # Radio proporcional a bicis disponibles
        radius = row['bikes'] * 1.5
        
        # Color segun porcentaje
        if row['available_pct'] < 30:
            color = 'red'
        elif row['available_pct'] < 60:
            color = 'orange'
        else:
            color = 'green'
        
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=radius,
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.6,
            popup=f"{row['name']}: {row['bikes']} bicis",
            tooltip=row['name']
        ).add_to(m)
    
    m

---

## 4. Marker Cluster

In [8]:
if FOLIUM_AVAILABLE:
    m = folium.Map(location=madrid_coords, zoom_start=13, tiles='CartoDB positron')
    
    # Crear cluster
    marker_cluster = MarkerCluster().add_to(m)
    
    # Anadir marcadores al cluster
    for idx, row in stations.iterrows():
        folium.Marker(
            location=[row['lat'], row['lon']],
            popup=f"{row['name']}: {row['bikes']} bicis",
            icon=folium.Icon(color='blue', icon='bicycle', prefix='fa')
        ).add_to(marker_cluster)
    
    m

---

## 5. Mapa de calor

In [9]:
if FOLIUM_AVAILABLE:
    # Generar puntos aleatorios (viajes)
    np.random.seed(42)
    n_trips = 500
    
    trips_data = []
    for _ in range(n_trips):
        # Punto aleatorio cerca de Madrid centro
        lat = 40.42 + np.random.normal(0, 0.015)
        lon = -3.70 + np.random.normal(0, 0.015)
        trips_data.append([lat, lon])
    
    # Mapa de calor
    m = folium.Map(location=madrid_coords, zoom_start=14, tiles='CartoDB dark_matter')
    
    HeatMap(
        trips_data,
        min_opacity=0.3,
        radius=15,
        blur=10
    ).add_to(m)
    
    m

---

## 6. Capas y control de capas

In [10]:
if FOLIUM_AVAILABLE:
    m = folium.Map(location=madrid_coords, zoom_start=14)
    
    # Capa 1: Estaciones activas
    active_layer = folium.FeatureGroup(name='Activas')
    for idx, row in stations[stations['status'] == 'active'].iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=10,
            color='green',
            fill=True,
            popup=row['name']
        ).add_to(active_layer)
    active_layer.add_to(m)
    
    # Capa 2: Estaciones con baja disponibilidad
    low_layer = folium.FeatureGroup(name='Baja disponibilidad')
    for idx, row in stations[stations['status'] == 'low'].iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=10,
            color='red',
            fill=True,
            popup=row['name']
        ).add_to(low_layer)
    low_layer.add_to(m)
    
    # Control de capas
    folium.LayerControl().add_to(m)
    
    m

---

## 7. Lineas y rutas

In [11]:
if FOLIUM_AVAILABLE:
    m = folium.Map(location=madrid_coords, zoom_start=14, tiles='CartoDB positron')
    
    # Ruta ejemplo: Sol -> Atocha -> Retiro
    route_coords = [
        [40.4168, -3.7038],  # Sol
        [40.4120, -3.6980],  # intermedio
        [40.4065, -3.6893],  # Atocha
        [40.4100, -3.6860],  # intermedio
        [40.4153, -3.6844]   # Retiro
    ]
    
    # Dibujar ruta
    folium.PolyLine(
        route_coords,
        weight=4,
        color='blue',
        opacity=0.8
    ).add_to(m)
    
    # Marcadores inicio/fin
    folium.Marker(route_coords[0], popup='Inicio: Sol',
                  icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(route_coords[-1], popup='Fin: Retiro',
                  icon=folium.Icon(color='red')).add_to(m)
    
    m

---

## 8. Guardar mapa

In [12]:
if FOLIUM_AVAILABLE:
    # Guardar como HTML
    # m.save('mapa_estaciones.html')
    
    print("Para guardar:")
    print("  m.save('mapa.html')")
    print("\nEl archivo HTML se puede abrir en cualquier navegador.")

Para guardar:
  m.save('mapa.html')

El archivo HTML se puede abrir en cualquier navegador.


---

## Resumen

| Elemento | Metodo |
|----------|--------|
| Mapa base | `folium.Map()` |
| Marcador | `folium.Marker()` |
| Circulo | `folium.CircleMarker()` |
| Linea | `folium.PolyLine()` |
| Heatmap | `HeatMap()` |
| Cluster | `MarkerCluster()` |
| Capas | `FeatureGroup()` + `LayerControl()` |

**Tiles populares:**
- `OpenStreetMap` (default)
- `CartoDB positron` (claro)
- `CartoDB dark_matter` (oscuro)
- `Stamen Terrain` (relieve)

---

**Anterior:** [08.02 - Operaciones Espaciales](08_02_spatial_operations.ipynb)  
**Siguiente:** Continuar con otros temas...